# Calculate hit_rate
## Model: EBM models (trained on tabular LDA features)
## Ranking data: knn_lda_ranking_v2
- Input: Top20 recommendation results by ebm_location, ebm_side_info, ebm_extended models
- Hit rate: Evaluated on different benchmarks: knn_lda_ranking_data, random_ranking_data, actual test_interaction data
- Hit rate @5, @10, @20 for each model

In [1]:
import numpy as np
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /home/jovyan/1_UT THESIS/CB12_MAIN/nb_recsys_ebm


In [4]:
# Load evaluation data
evaluation_data = pd.read_csv('./nb_recsys_ebm/ranking_data_knn_lda_v2.csv')

In [5]:
evaluation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369098 entries, 0 to 369097
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   UserID   369098 non-null  int64  
 1   JobID    369098 non-null  int64  
 2   label    369098 non-null  float64
 3   City     369098 non-null  float64
 4   State    369098 non-null  float64
 5   Country  369098 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 16.9 MB


In [6]:
evaluation_data.label.value_counts()

label
0.0    368951
1.0       147
Name: count, dtype: int64

In [7]:
# Load random sample data
ranking_data_random = pd.read_csv('./nb_recsys_ebm/ranking_data_random.csv')
ranking_data_random.label.value_counts()

label
0    367884
1      3716
Name: count, dtype: int64

In [8]:
models = []
benchmarks = []
hits_5 = []
hits_10 = []
hits_20 = []

## Load recommendation results

In [10]:
ebm_location_recommendations = pd.read_csv('./output_topN_ebm/rec_result_location_knn.csv')
ebm_extended_recommendations = pd.read_csv('./output_topN_ebm/rec_result_extended_knn.csv')
ebm_side_info_recommendations = pd.read_csv('./output_topN_ebm/rec_result_side_info_knn.csv')

In [11]:
def get_hit_rate_at_k(recommendations, rankings, K):
    total_users = len(recommendations['UserID'].unique())
    hit_count = 0
    # mark_stop = 0
    for user_id in recommendations['UserID'].unique():
        user_recommendations = recommendations[recommendations['UserID'] == user_id]['JobID'].tolist()[:K]
        actual_job_ids = rankings[(rankings['UserID'] == user_id) & (rankings['label'] == 1)]['JobID'].tolist()

        for rec_job_id in user_recommendations:
            if rec_job_id in actual_job_ids:
                # print('hit detail UserID, JobID: ', user_id, rec_job_id)
                hit_count += 1
                break     
        # mark_stop =+1
        # if mark_stop == 2:
        #     break
    print('final hit_count:', hit_count)
    hit_rate = hit_count / total_users
    return hit_rate

# Calculate hit_rate (manual) - Benchmark - random sample data
Positive items: Only consider label 1

In [12]:
benchmark = 'ranking_data_random'

## Model EBM_location

In [13]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 18
Model: ebm_location - Hit rate at 5: 0.004876727174207532
final hit_count: 21
Model: ebm_location - Hit rate at 10: 0.005689515036575454
final hit_count: 22
Model: ebm_location - Hit rate at 20: 0.005960444324031428
CPU times: user 13.3 s, sys: 0 ns, total: 13.3 s
Wall time: 13.3 s


In [14]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Model: ebm_side_info

In [15]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 2
Model: ebm_side_info - Hit rate at 5: 0.000541858574911948
final hit_count: 5
Model: ebm_side_info - Hit rate at 10: 0.00135464643727987
final hit_count: 8
Model: ebm_side_info - Hit rate at 20: 0.002167434299647792
CPU times: user 13.2 s, sys: 281 ms, total: 13.5 s
Wall time: 13.5 s


In [16]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Model: ebm_extended

In [17]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 18
Model: ebm_extended - Hit rate at 5: 0.004876727174207532
final hit_count: 21
Model: ebm_extended - Hit rate at 10: 0.005689515036575454
final hit_count: 22
Model: ebm_extended - Hit rate at 20: 0.005960444324031428
CPU times: user 12.8 s, sys: 17.9 ms, total: 12.8 s
Wall time: 12.8 s


In [18]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Calculate hit_rate (manual) - Benchmark - evaluation data
Positive items: Only consider label 1

In [19]:
benchmark = 'ranking_knn_lda'

### Model: ebm_location

In [20]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 3
Model: ebm_location - Hit rate at 5: 0.000812787862367922
final hit_count: 3
Model: ebm_location - Hit rate at 10: 0.000812787862367922
final hit_count: 3
Model: ebm_location - Hit rate at 20: 0.000812787862367922
CPU times: user 12.7 s, sys: 12.9 ms, total: 12.7 s
Wall time: 12.7 s


In [21]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: ebm_side_info

In [22]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 1
Model: ebm_side_info - Hit rate at 5: 0.000270929287455974
final hit_count: 2
Model: ebm_side_info - Hit rate at 10: 0.000541858574911948
final hit_count: 3
Model: ebm_side_info - Hit rate at 20: 0.000812787862367922
CPU times: user 11.8 s, sys: 6.1 ms, total: 11.8 s
Wall time: 11.8 s


In [23]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: ebm_extended

In [24]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 3
Model: ebm_extended - Hit rate at 5: 0.000812787862367922
final hit_count: 4
Model: ebm_extended - Hit rate at 10: 0.001083717149823896
final hit_count: 6
Model: ebm_extended - Hit rate at 20: 0.001625575724735844
CPU times: user 11.8 s, sys: 10.9 ms, total: 11.9 s
Wall time: 11.9 s


In [25]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

# Calculate hit_rate (manual) - Evaluation data: test_interaction data

In [26]:
benchmark = 'actual_test_interaction'

In [27]:
# Load all interaction data (LDA tabular with side information)
train_data_extended = pd.read_csv('./xai_recsys/train_data_extended.csv')
test_data_extended = pd.read_csv('./xai_recsys/test_data_extended.csv')

In [28]:
# Get all interaction data
select_cols = ['UserID', 'JobID', 'label','Split']
actual_interaction_train = train_data_extended[select_cols]
actual_interaction_test = test_data_extended[select_cols]
actual_interaction_df = pd.concat([actual_interaction_train,actual_interaction_test])

In [29]:
len(actual_interaction_train), len(actual_interaction_test)

(563889, 15736)

### Model: EBM_location

In [30]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 28
Model: ebm_location - Hit rate at 5 - evaluated on actual interaction (test): 0.007586020048767272
final hit_count: 33
Model: ebm_location - Hit rate at 10 - evaluated on actual interaction (test): 0.008940666486047142
final hit_count: 34
Model: ebm_location - Hit rate at 20 - evaluated on actual interaction (test): 0.009211595773503115
CPU times: user 7.78 s, sys: 11.2 ms, total: 7.79 s
Wall time: 7.78 s


In [31]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM_side_info

In [32]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 4
Model: ebm_side_info - Hit rate at 5 - evaluated on actual interaction (test): 0.001083717149823896
final hit_count: 8
Model: ebm_side_info - Hit rate at 10 - evaluated on actual interaction (test): 0.002167434299647792
final hit_count: 13
Model: ebm_side_info - Hit rate at 20 - evaluated on actual interaction (test): 0.003522080736927662
CPU times: user 6.94 s, sys: 3.37 ms, total: 6.94 s
Wall time: 6.94 s


In [33]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM_extended

In [34]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 27
Model: ebm_extended - Hit rate at 5 - evaluated on actual interaction (test): 0.007315090761311298
final hit_count: 32
Model: ebm_extended - Hit rate at 10 - evaluated on actual interaction (test): 0.008669737198591168
final hit_count: 35
Model: ebm_extended - Hit rate at 20 - evaluated on actual interaction (test): 0.00948252506095909
CPU times: user 6.95 s, sys: 7.5 ms, total: 6.95 s
Wall time: 6.95 s


In [35]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Calculate hit_rate (manual) - Evaluation data: all interaction data

In [36]:
benchmark = 'all_interaction_data'

### Model: EBM_location

In [37]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 28
Model: ebm_location - Hit rate at 5 - evaluated on actual interaction (all): 0.007586020048767272
final hit_count: 33
Model: ebm_location - Hit rate at 10 - evaluated on actual interaction (all): 0.008940666486047142
final hit_count: 34
Model: ebm_location - Hit rate at 20 - evaluated on actual interaction (all): 0.009211595773503115
CPU times: user 14.8 s, sys: 22.4 ms, total: 14.8 s
Wall time: 14.8 s


In [38]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: side_info

In [39]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 4
Model: ebm_side_info - Hit rate at 5 - evaluated on actual interaction (all): 0.001083717149823896
final hit_count: 8
Model: ebm_side_info - Hit rate at 10 - evaluated on actual interaction (all): 0.002167434299647792
final hit_count: 13
Model: ebm_side_info - Hit rate at 20 - evaluated on actual interaction (all): 0.003522080736927662
CPU times: user 14.8 s, sys: 7.07 ms, total: 14.8 s
Wall time: 14.8 s


In [40]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM - extended

In [41]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 27
Model: ebm_extended - Hit rate at 5 - evaluated on actual interaction (all): 0.007315090761311298
final hit_count: 32
Model: ebm_extended - Hit rate at 10 - evaluated on actual interaction (all): 0.008669737198591168
final hit_count: 35
Model: ebm_extended - Hit rate at 20 - evaluated on actual interaction (all): 0.00948252506095909
CPU times: user 14.8 s, sys: 2.84 ms, total: 14.8 s
Wall time: 14.8 s


In [42]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

# Summary

In [43]:
len(models), len(benchmarks), len(hits_5), len(hits_10), len(hits_20)

(12, 12, 12, 12, 12)

In [44]:
ebm_hit = pd.DataFrame(columns = ["model","hit_5", "hit_10", "hit_20", 'benchmark'])
ebm_hit.model = models
ebm_hit.benchmark = benchmarks
ebm_hit.hit_5 = hits_5
ebm_hit.hit_10 = hits_10
ebm_hit.hit_20 = hits_20

In [45]:
ebm_hit

,model,hit_5,hit_10,hit_20,benchmark
0,ebm_location,0.004877,0.005690,0.005960,ranking_data_random
1,ebm_side_info,0.000542,0.001355,0.002167,ranking_data_random
2,ebm_extended,0.004877,0.005690,0.005960,ranking_data_random
3,ebm_location,0.000813,0.000813,0.000813,ranking_knn_lda
4,ebm_side_info,0.000271,0.000542,0.000813,ranking_knn_lda
5,ebm_extended,0.000813,0.001084,0.001626,ranking_knn_lda
6,ebm_location,0.007586,0.008941,0.009212,actual_test_interaction
7,ebm_side_info,0.001084,0.002167,0.003522,actual_test_interaction
8,ebm_extended,0.007315,0.008670,0.009483,actual_test_interaction
9,ebm_location,0.007586,0.008941,0.009212,all_interaction_data


In [46]:
ebm_hit.to_csv('./nb_recsys_ebm/ebm_hit_knn.csv', header=True, index=False)

# Data-centric explanation
- Count number of JobID for positive interaction in test data

In [ ]:
summary_test_interaction = actual_interaction_test.groupby('UserID').agg(Count_JobID=('JobID', 'count')).reset_index()

In [ ]:
summary_test_interaction.sort_values(by = 'Count_JobID', ascending=False)

In [ ]:
summary_test_interaction['Count_JobID'].value_counts()

In [ ]:
summary_positive_test_interaction = actual_interaction_test[actual_interaction_test.label==1].groupby('UserID').agg(Count_JobID=('JobID', 'count')).reset_index()

In [ ]:
summary_positive_test_interaction.sort_values(by = 'Count_JobID', ascending=False)

In [ ]:
summary_positive_test_interaction['Count_JobID'].value_counts()